# **Projeto de Bioinformática - Descoberta Computacional de Fármacos [Parte 3] Cálculo do Descritor e Preparação do Conjunto de Dados**

Juan Oliveira

Na **Parte 3**, calcularemos descritores moleculares que são descrições essencialmente quantitativas dos compostos no conjunto de dados. Por fim, prepararemos isso em um conjunto de dados para construção de modelo subsequente na Parte 4.

---

## **Download PaDEL-Descriptor**

O PaDEL-Descriptor é um software amplamente utilizado para calcular descritores moleculares em química computacional e quiminformática. Os descritores moleculares são representações numéricas que capturam várias propriedades físicas e químicas de uma molécula, permitindo a análise e comparação de compostos químicos.

O nome "PaDEL" é uma abreviação de "Predictive Activity Modeling based on Descriptor Labels," e o software foi desenvolvido para auxiliar na criação de modelos de previsão de atividade biológica de compostos químicos. Ele fornece uma ampla gama de descritores moleculares que podem ser usados para caracterizar uma molécula com base em suas estruturas químicas.

Alguns exemplos de descritores que o PaDEL-Descriptor pode calcular incluem:

Descritores de propriedades físico-químicas: Eles abrangem propriedades como massa molecular, polaridade, solubilidade, energia de ionização e muitos outros.

Descritores de conectividade: Eles descrevem a conectividade entre os átomos na molécula, incluindo informações sobre anéis aromáticos, cadeias carbônicas e grupos funcionais.

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

--2024-07-01 22:20:02--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2024-07-01 22:20:03--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M  48.4MB/s    in 0.5s    

2024-07-01 22:20:05 (48.4 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2024-07-01 22:20:05--  https://git

Descompactação dos arquivos que irao ser utilizados para o funcionamento dos programas

In [ ]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

Baixe os dados de bioatividade do ChEMBL, que foram pré-processados a partir das partes 1 e 2 desta série de Projetos de Bioinformática. Aqui, estaremos usando o arquivo **bioactivity_data_3class_pIC50.csv**, que essencialmente contém os valores de pIC50 que utilizaremos para construir um modelo de regressão.

In [ ]:
! wget https://raw.githubusercontent.com/dataprofessor/data/master/acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv

--2024-07-01 22:20:07--  https://raw.githubusercontent.com/dataprofessor/data/master/acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 655414 (640K) [text/plain]
Saving to: ‘acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv’

acetylcholinesteras 100%[===================>] 640.05K  3.61MB/s    in 0.2s    

2024-07-01 22:20:08 (3.61 MB/s) - ‘acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv’ saved [655414/655414]



Para importar esses dados para o banco de dados, primeiro foi feito o carregamento do pacote pandas e, em seguida, o nome ‘df3’ foi atribuído a esse banco de dados.

In [ ]:
import pandas as pd

Salvar em csv e visualizar a planilha

In [ ]:
from google.colab import drive # Importando o módulo drive do pacote google.colab para montar o Google Drive
drive.mount('/content/gdrive', force_remount=True) # Definindo o caminho do projeto no Google Drive

proj_path = '/content/gdrive/MyDrive/Colab Notebooks/' # Caminho do projeto no Google Drive
file_name4='VHC_05_bioactivity_data_2class_pIC50.csv' ## nome do arquivo
df3 = pd.read_csv(proj_path + file_name4) ## visualizando o arquivo csv
df3

Mounted at /content/gdrive


O **"Canonical SMILES" (Simplified Molecular Input Line Entry System)** é uma forma padronizada de representar estruturas moleculares quimicamente, principalmente para fins de armazenamento, recuperação e manipulação de informações sobre compostos químicos. É uma notação textual que descreve a estrutura de uma molécula por meio da disposição de seus átomos e ligações usando caracteres alfanuméricos.

A representação do Canonical SMILES segue algumas regras específicas:

1. Os átomos de hidrogênio geralmente são omitidos, a menos que sejam importantes para a representação da estrutura ou da carga.

2. Ligações simples são frequentemente representadas apenas por meio do uso de hífens (-), e ligações duplas e triplas são indicadas por meio de símbolos (=) e (#), respectivamente.

3. Anéis são representados por números, indicando quais átomos fazem parte do anel.

4. Grupos funcionais e outros substituintes também são representados por meio de notações específicas.

A vantagem do Canonical SMILES é que ele fornece uma representação concisa e única das estruturas moleculares, o que facilita a comparação, pesquisa e análise de compostos químicos em bancos de dados e na modelagem molecular.

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id'] ## Foi pegos os dados da coluna de forma molecular assim como sua identificação
df3_selection = df3[selection] ## selecionando as colunas
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False) ## salvando em um arquivo smi

In [ ]:
! cat molecule.smi | head -5 ## visualizando o arquivo smi

CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2)CN1C(=O)[C@H](NC(=O)[C@H](NC(=O)c1cnccn1)C(C)C)C(C)C)C(=O)C(=O)NCCc1ccccc1	CHEMBL32704
CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2)CN1C(=O)[C@H](NC(=O)[C@H](NC(=O)c1cnccn1)C(C)C)C(C)C)C(=O)C(=O)NCc1ccccc1	CHEMBL33248
CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2)CN1C(=O)[C@H](NC(=O)[C@H](NC(=O)c1cnccn1)C(C)C)C(C)C)C(=O)C(=O)N[C@@H](C)c1ccccc1	CHEMBL285069
CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2)CN1C(=O)[C@H](NC(=O)[C@H](NC(=O)c1cnccn1)C(C)C)C(C)C)C(=O)C(=O)NC1CC1	CHEMBL286124
CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2)CN1C(=O)[C@H](NC(=O)[C@H](NC(=O)c1cnccn1)C(C)C)C(C)C)C(=O)C(=O)c1ccccc1	CHEMBL285686


Essa parte mostra quantas moleculas foram pegas nessa coluna

In [ ]:
! cat molecule.smi | wc -l ## conta as linhas do arquivo smi

572


## **Calcular descritores de fingerprint**


Descritores de fingerprint são valores numéricos que representam características específicas de uma molécula com base em sua estrutura química. Esses descritores são usados para quantificar propriedades moleculares e auxiliar em pesquisas farmacológicas, como a previsão de atividade biológica de compostos

### **Calculate PaDEL descriptors**

In [ ]:
! cat padel.sh ## visualizando o arquivo sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv


Realizando uma limpeza nas estruturas químicas para remover impurezas. Essa limpeza pode envolver a exclusão de compostos como sais (como sódio e cloreto) e pequenos ácidos orgânicos. O objetivo é garantir que os dados sejam mais precisos e relevantes para análises subsequentes.

In [ ]:
! bash padel.sh ## rodando o arquivo sh

Processing CHEMBL32704 in molecule.smi (1/572). 
Processing CHEMBL33248 in molecule.smi (2/572). 
Processing CHEMBL286124 in molecule.smi (4/572). Average speed: 4.37 s/mol.
Processing CHEMBL285069 in molecule.smi (3/572). Average speed: 8.48 s/mol.
Processing CHEMBL285686 in molecule.smi (5/572). Average speed: 3.91 s/mol.
Processing CHEMBL284771 in molecule.smi (6/572). Average speed: 3.12 s/mol.
Processing CHEMBL423300 in molecule.smi (7/572). Average speed: 2.90 s/mol.
Processing CHEMBL285260 in molecule.smi (8/572). Average speed: 2.89 s/mol.
Processing CHEMBL116118 in molecule.smi (9/572). Average speed: 2.56 s/mol.
Processing CHEMBL116118 in molecule.smi (10/572). Average speed: 2.35 s/mol.
Processing CHEMBL116118 in molecule.smi (11/572). Average speed: 2.44 s/mol.
Processing CHEMBL116118 in molecule.smi (12/572). Average speed: 2.24 s/mol.
Processing CHEMBL111936 in molecule.smi (13/572). Average speed: 2.04 s/mol.
Processing CHEMBL111936 in molecule.smi (15/572). Average spee

Para ver se esta lendo o arquivo...

In [ ]:
! ls -l ## verificando se esta lendo o arquivo

total 26916
-rw-r--r-- 1 root root   655414 Jul  1 22:20 acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv
-rw-r--r-- 1 root root  1028273 Jul  1 22:26 descriptors_output.csv
drwx------ 6 root root     4096 Jul  1 22:20 gdrive
drwxr-xr-x 3 root root     4096 Jul  1 22:20 __MACOSX
-rw-r--r-- 1 root root    76754 Jul  1 22:20 molecule.smi
drwxrwxr-x 4 root root     4096 May 30  2020 PaDEL-Descriptor
-rw-r--r-- 1 root root      231 Jul  1 22:20 padel.sh
-rw-r--r-- 1 root root 25768637 Jul  1 22:20 padel.zip
drwxr-xr-x 1 root root     4096 Jun 28 13:24 sample_data


## **Preparando as Matrizes de Dados X e Y**

"É comum separar as matrizes entre dados X e y no contexto de aprendizado de máquina e análise de dados.

**Variáveis independentes (X):** Essas representam as características ou atributos usados para prever ou explicar uma variável de interesse. Por exemplo, ao construir um modelo para prever o preço de casas, as variáveis independentes podem incluir o tamanho da casa, o número de quartos e a localização. Essas informações são organizadas em uma matriz X.

**Variável dependente (y):** Essa é a variável que desejamos prever ou explicar. No caso das casas, a variável dependente seria o preço da casa, e essa informação é armazenada em um vetor y.

A separação entre X e y é importante por algumas razões:

**Treinamento do modelo:** Durante o treinamento de um modelo de aprendizado de máquina, utilizamos os dados em X para aprender a relação entre as variáveis independentes e dependentes. O modelo ajusta seus parâmetros com base nesses dados.
**Teste e avaliação:** Após o treinamento, é necessário testar o modelo em dados não vistos. Para isso, separamos uma parte dos dados originais para formar um conjunto de teste. Os dados em X_test são usados para fazer previsões, e comparamos essas previsões com os valores reais em y_test para avaliar o desempenho do modelo.

**Generalização:** A separação entre treinamento e teste ajuda a avaliar a capacidade do modelo de generalizar para novos dados. Se utilizássemos todos os dados para treinamento, o modelo poderia se ajustar excessivamente aos dados específicos do conjunto de treinamento e não se sair bem em dados desconhecidos."

### **X data matrix**

In [ ]:
df3_X = pd.read_csv('descriptors_output.csv') ## visualizando o arquivo csv

In [ ]:
df3_X ## visualizando o arquivo

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,CHEMBL33248,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL32704,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL286124,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL285069,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL285686,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,CHEMBL3126842,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
568,CHEMBL3126842,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
569,CHEMBL3126842,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
570,CHEMBL3126842,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df3_X = df3_X.drop(columns=['Name'])## tiramos a coluna 'name', pois precisamos de dados quantitativos...
df3_X ## visualizando a matriz

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
568,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
569,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
570,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## **Variavel Y**

### **Converter IC50 para pIC50**
"Converter IC50 em pIC50" significa transformar os valores de IC50 (meia concentração inibitória) em valores de pIC50. O pIC50 é uma transformação matemática comumente usada para melhorar a interpretação dos dados de atividade biológica.

O IC50 é uma medida da concentração de um composto químico que é necessária para inibir uma determinada atividade biológica em 50%. Quanto menor o valor de IC50, mais potente é o composto.

O pIC50 é calculado da seguinte forma:

pIC50 = -log(IC50)

Essa transformação converte o IC50 em uma escala logarítmica negativa, o que torna os valores mais convenientes para análise, especialmente quando se lida com uma ampla gama de potências de compostos. Geralmente, valores de pIC50 maiores correspondem a compostos mais potentes.

A conversão para pIC50 é especialmente útil ao comparar e analisar dados de atividade biológica, pois simplifica a interpretação e permite uma representação mais clara da relação entre a concentração de um composto e sua atividade inibitória. Portanto, ao "converter IC50 em pIC50", você está aplicando essa transformação matemática aos dados para facilitar sua análise e interpretação.

In [ ]:
df3_Y = df3['pIC50'] ## selecionando a coluna pIC50
df3_Y ## visualizando a coluna

0      6.283997
1      6.657577
2      6.657577
3      6.376751
4      7.585027
         ...   
567    6.785156
568    8.744727
569    7.701147
570    8.318759
571    7.210419
Name: pIC50, Length: 572, dtype: float64

## **Combinando X e Y varriaveis**

In [ ]:
dataset3 = pd.concat([df3_X,df3_Y], axis=1) ## concatenando as colunas
dataset3 ## visualizando a matriz

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.283997
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.657577
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.657577
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.376751
4,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.585027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.785156
568,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.744727
569,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.701147
570,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,8.318759


In [ ]:
file_name5 = 'VHC_06_bioactivity_data_3class_pIC50_pubchem_fp.csv' ## nome do arquivo
dataset3.to_csv(proj_path + file_name5, index=False) ## salvando a planilha em um arquivo csv

Juntando as duas tabelas df3 (vc sabe)

In [ ]:
df3 = df3.drop(columns=['pIC50', 'Unnamed: 0']) # Removendo as colunas 'pIC50' e 'Unnamed: 0' do DataFrame df3
dffinal = pd.concat([df3,df3_X,df3_Y], axis=1) # Concatenando os DataFrames df3, df3_X e df3_Y ao longo do eixo 1 (colunas)
dffinal ## visualizando a planilha

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,PubchemFP0,PubchemFP1,PubchemFP2,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,CHEMBL32704,CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2...,active,824.980,2.7491,4.0,10.0,1,1,1,...,0,0,0,0,0,0,0,0,0,6.283997
1,CHEMBL33248,CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2...,active,810.953,2.7066,4.0,10.0,1,1,1,...,0,0,0,0,0,0,0,0,0,6.657577
2,CHEMBL285069,CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2...,active,824.980,3.2676,4.0,10.0,1,1,1,...,0,0,0,0,0,0,0,0,0,6.657577
3,CHEMBL286124,CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2...,active,760.893,1.6688,4.0,10.0,1,1,1,...,0,0,0,0,0,0,0,0,0,6.376751
4,CHEMBL285686,CCCC(NC(=O)[C@@H]1C[C@@H](OC(=O)N2CCc3ccccc3C2...,active,781.911,3.2731,3.0,10.0,1,1,1,...,0,0,0,0,0,0,0,0,0,7.585027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,CHEMBL3126842,COc1ccc2c(c1)[C@@H]1C[C@]1(C(=O)N1C3CCC1CN(C)C...,active,659.853,4.6931,1.0,7.0,1,1,1,...,0,0,0,0,0,0,0,0,0,6.785156
568,CHEMBL3126842,COc1ccc2c(c1)[C@@H]1C[C@]1(C(=O)N1C3CCC1CN(C)C...,active,659.853,4.6931,1.0,7.0,1,1,1,...,0,0,0,0,0,0,0,0,0,8.744727
569,CHEMBL3126842,COc1ccc2c(c1)[C@@H]1C[C@]1(C(=O)N1C3CCC1CN(C)C...,active,659.853,4.6931,1.0,7.0,1,1,1,...,0,0,0,0,0,0,0,0,0,7.701147
570,CHEMBL3126842,COc1ccc2c(c1)[C@@H]1C[C@]1(C(=O)N1C3CCC1CN(C)C...,active,659.853,4.6931,1.0,7.0,1,1,1,...,0,0,0,0,0,0,0,0,0,8.318759


Salvando a planilha em um arquivo csv

In [ ]:
file_name7 = 'VHC_07_bioactivity_data_3class_pIC50_pubchem_fp.csv' ## nome do arquivo
dffinal.to_csv(proj_path + file_name7, index=False) ## salvando a planilha em um arquivo csv